## SkilltreeのJson作成

In [4]:
import pandas as pd
csv_jp = pd.read_csv('SkillListJp.tsv',sep='\t')
csv_en = pd.read_csv('SkillListEn.tsv',sep='\t')
csv_cn = pd.read_csv('SkillListCn.tsv',sep='\t')
skill_tree = 'ABCDEFGHIJKLMNOPQRSTUVWXYZ'
skill_key = 'abcdefghijklmnopqrstuvwxyz'
category_keys = ['基本', '戦闘', '魔法', '生活', 'トリック']
category_keys_en = ['Basic', 'Combat', 'Magic', 'Life', 'Trick']
category_keys_cn = ['基础', '战斗', '魔法', '生活', '陷阱']
tier_keys = ['Tier1', 'Tier2', 'Tier3', 'Tier4', 'Tier5']
tier_limit = {'基本': [3, 5, 7, 10, 99], '戦闘': [3, 6, 12, 12, 99], '魔法': [3, 6, 12, 12, 99], '生活': [3, 6, 12, 12, 99], 'トリック': [3, 6, 12, 12, 99]}
icon_path = './img/:name.png'
skilltree_ver = 'v1'

In [5]:
# aa, abなどのスキルキーを作成
def CreateSkillKey(index):
    return (skill_key[int(index/26)] + skill_key[int(index%26)])

In [6]:
result = []
tree_index = 0

for category_key in category_keys:
    result.append({
        'name': {
            'jp': category_key,
            'en': category_keys_en[tree_index],
            'cn': category_keys_cn[tree_index],
        },
        'key': skill_tree[tree_index],
        'tiers': []
    })
    
    tier_index = 0
    for tier_key in tier_keys:
        result[tree_index]['tiers'].append({
            'limit': tier_limit[category_key][tier_index],
            'skills': []
        })

        tier_data = csv_jp[(csv_jp['CategoryName'] == category_key) & (csv_jp['Tier'] == tier_key) & (csv_jp['Tier'] != 'None')].sort_values('Tier', ascending=True).iterrows()
        skill_index = 0
        for skill in tier_data:
            skill_en = csv_en[csv_en['ID'] == skill[1]['ID']] # 英語tsvから同一スキルの情報を取得
            skill_cn = csv_cn[csv_cn['ID'] == skill[1]['ID']]
            result[tree_index]['tiers'][tier_index]['skills'].append({
                'id': skill[1]['ID'],
                'key': CreateSkillKey(skill_index),
                'name': {
                    'jp': skill[1]['SkillName'],
                    'en': skill_en['SkillName'].iloc[-1],
                    'cn': skill_cn['SkillName'].iloc[-1]
                },
                'max': skill[1]['MaxLevel'],
                'description': {
                    'jp': skill[1]['Description'],
                    'en': skill_en['Description'].iloc[-1],
                    'cn': skill_cn['Description'].iloc[-1]
                },
                'icon': skill_en['IconName'].iloc[-1] + '.png'
            })
            skill_index+=1
        tier_index+=1
        
    tree_index+=1

In [7]:
import json
with open(skilltree_ver + '.json', 'w', encoding='utf-8') as f:
    json.dump(result, f, ensure_ascii=False)